In [2]:
source("./EMR_LIP.R")
library(readxl)


In [3]:
ds <- read_excel("data.xlsx", col_names=T, sheet=1)
var_dict <- read_excel("var_dict.xlsx", col_names=T, sheet=1)
var_dict1 <- var_dict[var_dict$time_type == "single", ,drop=F]
var_dict2 <- var_dict[var_dict$time_type == "interval", ,drop=F]
ds_map <- read_excel("var_dict.xlsx", col_names=T, sheet=2)

In [10]:
var_dict

itemid,time_type,value_type,agg_fun,fill1,fill2,cont,low,high,valid_value,operational,last_value
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<dbl>
A,single,num,mean,locb,lin,6,-1,18,NA,no,4.5
B,single,ord,mode,locb,locf,3,NA,NA,NA,no,NA
C,single,cat,mode,locb,locf,a,NA,NA,a|b,no,NA
D,single,bin,any,locb,zero,1,NA,NA,NA,no,NA
E,interval,num,sum_w,zero,zero,2,-1,18,NA,yes,2.5


In [5]:
head(ds, 10)

sid,time,time2,item_id,value
<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,1,NA,a,2
1,1,NA,b,1
1,1,NA,c,b
1,1,NA,d,0
1,1,NA,a,8
1,2,NA,b,2
1,3,NA,a,1
1,3,NA,b,2
1,3,NA,C,a


In [6]:
ds <- rename_long_table(ds, ds_map$old_name, ds_map$new_name, "item_id")

In [7]:
head(ds, 10)

sid,time,time2,item_id,value
<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,1,NA,A,2
1,1,NA,B,1
1,1,NA,C,b
1,1,NA,D,0
1,1,NA,A,8
1,2,NA,B,2
1,3,NA,A,1
1,3,NA,B,2
1,3,NA,C,a


In [8]:
ds <- remove_extreme_value_long(ds, var_dict$itemid, var_dict$value_type,
                                 "item_id", "value", var_dict)

In [9]:
head(ds, 10)

sid,time,time2,item_id,value
<dbl>,<dbl>,<dbl>,<chr>,<chr>
1,1,NA,A,2
1,1,NA,A,8
1,3,NA,A,1
1,3,NA,A,5
1,4,NA,A,1
1,4,NA,A,5
1,4,NA,A,1
1,4,NA,A,5
1,5,NA,A,1


In [11]:
stat_ds1 <- get_stat_long(ds, 
                          var_dict1$itemid, 
                          var_dict1$value_type,
                          "item_id",
                          "value",
                          var_dict1$cont)
stat_ds2 <- get_stat_long(ds, 
                          var_dict2$itemid, 
                          var_dict2$value_type,
                          "item_id",
                          "value",
                          var_dict2$cont)


In [12]:
ds_k <- ds[ds$sid == 1,]
ds_k1 <- resample_single_long(ds_k,
                                var_dict1$itemid,
                                var_dict1$value_type,
                                var_dict1$agg_fun,
                                1:15,
                                "item_id",
                                "value",
                                "time",
                                1,
                                direction = "both",
                                keepNArow = T,
                                keep_first = F)
# undebug(resample_single_long)
ind1 <- (ds_k1[,2] == "1")
ds_k1 <- ds_k1[,c(1,3:ncol(ds_k1))]
mask_k1 <- get_mask(ds_k1, 2:ncol(ds_k1), 1)
ds_k1 <- fill(ds_k1, 2:ncol(ds_k1), 1, get_type(stat_ds1), var_dict1$fill1, var_dict1$fill2, stat_ds1)
ds_k1 <- fill_last_values(ds_k1, mask_k1, 2:ncol(ds_k1), 1, var_dict1)
ds_k1 <- to_onehot(ds_k1, 2:ncol(ds_k1), 1, get_type(stat_ds1), stat_ds1)

In [13]:
head(ds_k1, 10)

time,A,B,C___1,C___2,D
1,5,1,0,1,0
2,4,2,0,1,0
3,3,3,1,0,1
4,3,2,1,0,1
5,3.66666666666667,3,1,0,1
6,2.33333333333333,3,1,0,0
7,1,2,1,0,0
8,1.4,2,1,0,0
9,1.8,3,1,0,1
10,2.2,3,1,0,0


In [14]:
ds_k2 <- resample_interval_long(ds_k,
                            var_dict2$itemid,
                            var_dict2$value_type,
                            var_dict2$agg_fun,
                            1:15,
                            "item_id",
                            "value",
                            "time",
                            "time2",
                            1,
                            direction = "both",
                            keepNArow = T,
                            keep_first = F)
ind2 <- (ds_k2[,2] == "1")
ds_k2 <- ds_k2[,c(1,3:ncol(ds_k2))]
mask_k2 <- get_mask(ds_k2, 2:ncol(ds_k2), 1)
ds_k2 <- fill(ds_k2, 2:ncol(ds_k2), 1, get_type(stat_ds2), var_dict2$fill1, var_dict2$fill2, stat_ds2)
ds_k2 <- fill_last_values(ds_k2, mask_k2, 2:ncol(ds_k2), 1, var_dict2)
ds_k2 <- to_onehot(ds_k2, 2:ncol(ds_k2), 1, get_type(stat_ds2), stat_ds2)

In [15]:
head(ds_k2, 10)

time,E
1,0.5
2,1.5
3,1.75
4,2
5,2.25
6,2.5
7,1.5
8,0
9,0
10,0


In [16]:
ds_k_ <- cbind(ds_k1, ds_k2[,2:ncol(ds_k2),drop=F]) %>% as.data.frame %>% lapply(., as.numeric) %>% as.data.frame
ds_k_ <- ds_k_[which(ind1 | ind2), ]

In [17]:
head(ds_k_, 10)

,time,A,B,C___1,C___2,D,E
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,5.000000,1,0,1,0,0.50
2,2,4.000000,2,0,1,0,1.50
3,3,3.000000,3,1,0,1,1.75
4,4,3.000000,2,1,0,1,2.00
5,5,3.666667,3,1,0,1,2.25
6,6,2.333333,3,1,0,0,2.50
7,7,1.000000,2,1,0,0,1.50
8,8,1.400000,2,1,0,0,0.00
9,9,1.800000,3,1,0,1,0.00
